# Saving The Graph

In [1]:
import tensorflow as tf
import os

In [2]:
tf.reset_default_graph()

In [3]:
checkpoint_dir = os.path.join('./experiment/', "checkpoints/")
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [4]:
with tf.variable_scope('step'):
    global_step_tensor = tf.Variable(0, trainable=False, name='global_step')
    global_step_input = tf.placeholder('int32', None, name='global_step_input')
    global_step_assign_op = global_step_tensor.assign(global_step_input)

In [5]:
saver = tf.train.Saver()

In [6]:
init = tf.global_variables_initializer()
sess = tf.Session()

In [7]:
sess.run(init)

In [8]:
sess.run(global_step_assign_op, {global_step_input: global_step_tensor.eval(sess)+1})
sess.run(global_step_assign_op, {global_step_input: global_step_tensor.eval(sess)+1})

2

In [9]:
saver.save(sess, checkpoint_dir, global_step_tensor)

'./experiment/checkpoints/-2'

In [10]:
sess.run(global_step_assign_op, {global_step_input: global_step_tensor.eval(sess)+1})

3

In [11]:
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
    print("Loading model checkpoint {}...\n".format(latest_checkpoint))
    saver.restore(sess, latest_checkpoint)

Loading model checkpoint ./experiment/checkpoints/-2...



In [12]:
print (global_step_tensor.eval(sess))

2


# tf.Gather vs tf.dynamic_partition

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.reset_default_graph()

In [3]:
batch_size = 5

In [4]:
predictions = tf.placeholder(dtype='float32', shape=[None, 4])
actions = tf.placeholder(dtype='int32', shape=[None])

In [6]:
gather_indices = tf.range(batch_size) * tf.shape(predictions)[1] + actions
gather = tf.gather(tf.reshape(predictions, [-1]), gather_indices)

In [7]:
partition_indices = tf.one_hot(actions, 4, dtype='int32')
dyn_partitions = tf.dynamic_partition(predictions, partition_indices, 2)

In [8]:
init = tf.global_variables_initializer()
sess = tf.Session()

In [9]:
sess.run(init)

In [10]:
data = np.reshape(np.array(range(20)), (batch_size, 4))
print(data)
taken_actions = np.random.choice(4, batch_size)
print (taken_actions)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]]
[1 2 1 2 0]


In [11]:
output = sess.run(dyn_partitions, {predictions: data, actions:taken_actions})

In [12]:
print(output[1])

[  1.   6.   9.  14.  16.]


In [13]:
output = sess.run(gather, {predictions: data, actions:taken_actions})

In [14]:
print(output)

[  1.   6.   9.  14.  16.]


# Histogram Summary

In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
tf.reset_default_graph()

In [3]:
summary_dir = os.path.join('./experiment/', "summary/")
if not os.path.exists(summary_dir):
    os.makedirs(summary_dir)

In [4]:
summary_input = tf.placeholder(tf.float32, [100, 200], name='hist_input')
summary_op = tf.summary.histogram(values=summary_input, name='summary_op')

In [5]:
init = tf.global_variables_initializer()
sess = tf.Session()

In [6]:
sess.run(init)

In [7]:
summary_writer = tf.summary.FileWriter(summary_dir, sess.graph)

In [8]:
data = np.random.randn(100, 200)
summary = sess.run(summary_op, {summary_input: data})

In [9]:
summary_writer.add_summary(summary, 1)
summary_writer.flush()

In [10]:
data = np.random.randn(100, 200) * 0.3
summary = sess.run(summary_op, {summary_input: data})

In [11]:
summary_writer.add_summary(summary, 2)
summary_writer.flush()

# Meta-data Summary

In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
tf.reset_default_graph()

In [3]:
summary_dir = os.path.join('./experiment/', "summary/")
if not os.path.exists(summary_dir):
    os.makedirs(summary_dir)

In [4]:
X1 = tf.placeholder(tf.float32, name='x1')
X2 = tf.placeholder(tf.float32, name='x2')
mul_op = tf.multiply(x=X1, y=X2, name="multiply")
summary_op = tf.summary.scalar(tensor=mul_op, name='summary_op')

In [5]:
init = tf.global_variables_initializer()
sess = tf.Session()

In [6]:
sess.run(init)

In [7]:
summary_writer = tf.summary.FileWriter(summary_dir, sess.graph)

In [8]:
for i in range(10):
    x1 = np.random.randn()
    x2 = np.random.randn()
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    _, summary = sess.run([mul_op, summary_op],
                      feed_dict={X1: x1, X2: x2},
                      options=run_options,
                      run_metadata=run_metadata)
    summary_writer.add_run_metadata(run_metadata, 'step%d' % i)
    summary_writer.add_summary(summary, i)
    print('Adding run metadata for', i)

Adding run metadata for 0
Adding run metadata for 1
Adding run metadata for 2
Adding run metadata for 3
Adding run metadata for 4
Adding run metadata for 5
Adding run metadata for 6
Adding run metadata for 7
Adding run metadata for 8
Adding run metadata for 9
